# Manual corrections

## Coref

### Character name list corrections

In [48]:
# Removal, edit, addition dictionaries
removals = {
    15: [
        'a_Tardigrade',
        'seconds',
        'the_Earth',
        'day'
    ],
    18: [
        'Earth',
        'the_White_House',
        'The_Big_Surprise',
        'the_Fezzgen',
    ],
    29: [
        'the_Cheetah',
        'the_Pomeranian',
        'the_alpaca',
    ]
}

transformations = {
    15: {
        'Ok_Chad': 'Chad',
        'Screamed_Lilly': 'Lilly'
    }
}

additions = {
    18: [
        'Dillan',
        'Sam',
    ]
}

In [49]:
import os

# I/O
coref_chars_dirpath = '/usr0/home/mamille2/storyq/student_scifi/output/char_coref_chars/'
coref_stories_dirpath = '/usr0/home/mamille2/storyq/student_scifi/output/char_coref_stories/'
edited_coref_chars_dirpath = '/usr0/home/mamille2/storyq/student_scifi/output_edited/char_coref_chars/'
edited_coref_stories_dirpath = '/usr0/home/mamille2/storyq/student_scifi/output_edited/char_coref_stories/'

story_ids = [15, 18, 29]

for story_id in story_ids:
    char_list_fpath = os.path.join(coref_chars_dirpath, f'story{story_id}.chars')
    edited_char_list_fpath = os.path.join(edited_coref_chars_dirpath, f'story{story_id}.chars')
    
    # Load characters
    with open(char_list_fpath, 'r') as f:
        chars = f.read().splitlines()
    
    # Change character lists
    chars = [char for char in chars if not char in removals[story_id]]
    if story_id in transformations:
        chars = [transformations[story_id][char] if char in transformations[story_id] else char for char in chars]
    if story_id in additions:
        chars += additions[story_id]

    # Save out
    with open(edited_char_list_fpath, 'w') as f:
        for char in chars:
            f.write(f'{char}\n')

### Character story file corrections

In [53]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

for story_id in story_ids:
    coref_story_fpath = os.path.join(coref_stories_dirpath, f'story{story_id}.coref.csv')
    edited_story_fpath = os.path.join(edited_coref_stories_dirpath, f'story{story_id}.coref.csv')
    
    # Load coref'ed story
    coref_story_data = pd.read_csv(coref_story_fpath)
    
    # Change characters
    for char in removals[story_id]:
        coref_story_data['text_tokenized'] = [remove_char_tag(text, char) for text in coref_story_data['text_tokenized']]
        
    if story_id in transformations:
        for char in transformations[story_id]:
            coref_story_data['text_tokenized'] = [replace_char_tag(text, char, transformations[story_id][char]) \
                                                 for text in coref_story_data['text_tokenized']]
            
    # Add characters
    if story_id in additions:
        for char in additions[story_id]:
            coref_story_data['text_tokenized'] = [add_char_tag(text, char) for text in coref_story_data['text_tokenized']]

#     Save out
    coref_story_data.to_csv(edited_story_fpath, index=False)
    print(edited_story_fpath)

/usr0/home/mamille2/storyq/student_scifi/output_edited/char_coref_stories/story15.coref.csv
/usr0/home/mamille2/storyq/student_scifi/output_edited/char_coref_stories/story18.coref.csv
/usr0/home/mamille2/storyq/student_scifi/output_edited/char_coref_stories/story29.coref.csv


In [51]:
import re

def remove_char_tag(text, char):
    """ Remove character tag from text """
    char_mentions = re.findall(f'<character name="{char}">(.*?)</character>', text)
    newtext = text
    for mention in char_mentions:
        newtext = re.sub(f'<character name="{char}">{mention}</character>', mention, newtext)
    return newtext

def replace_char_tag(text, char, new_char):
    """ Replace character name in text """
    return text.replace(f'name="{char}"', f'name="{new_char}"')

def add_char_tag(text, char):
    return text.replace(char, f'<character name="{char}">{char}</character>')

In [27]:
test = 'The scorpion came at <character name="fred">fred</character> and <character name="fred">george</character> and they ran.'
remove_char_tag(test, 'fred')

'The scorpion came at fred and george and they ran.'

In [31]:
test = 'The scorpion came at <character name="fred">fred</character> and <character name="fred">george</character> and they ran.'
replace_char_tag(test, 'fred', 'jim')

'The scorpion came at <character name="jim">fred</character> and <character name="jim">george</character> and they ran.'

# Combine coref CSVs into one big CSV

In [54]:
# Load all coref CSVs
import os
import pandas as pd

combined = pd.DataFrame()
output_dirpath = '/usr0/home/mamille2/storyq/student_scifi/output_edited/char_coref_stories/'
selected_stories = [15, 18, 29]
selected_fpaths = [os.path.join(output_dirpath, f'story{story}.coref.csv') for story in selected_stories]
for fpath in selected_fpaths:
# for fname in os.listdir(output_dirpath):
#     if not fname.endswith('.csv'):
#         continue
#     fpath = os.path.join(output_dirpath, fname)
    data = pd.read_csv(fpath)
    combined = pd.concat([combined, data])
    
combined

# Save out
outpath = '/usr0/home/mamille2/storyq/student_scifi/output_edited/combined_coref.csv'
combined.to_csv(outpath, index=False)

# Focus on only protagonists/antagonists

## Merge filtered pipeline results into character features csv

In [45]:
# Load main character featurs
import pandas as pd

feats_path = '/home/mamille2/storyq/student_scifi/main_character_features.csv'
feats = pd.read_csv(feats_path)
feats['matched_characters_list'] = feats['matched_characters'].map(lambda x: eval(x))
feats

story_id                   Story     pipeline_story_name Main character  \
0          1                    3002                    3002           Joey   
1          1                    3002                    3002           Joel   
2          2  A leap into the future  A leap into the future          Timmy   
3          2  A leap into the future  A leap into the future           Jota   
4          2  A leap into the future  A leap into the future          Annie   
..       ...                     ...                     ...            ...   
89        35    Chickens strike back    Chickens Strike Back      Mcchicken   
90        35    Chickens strike back    Chickens Strike Back            Egg   
91        35    Chickens strike back    Chickens Strike Back          Rocco   
92        35    Chickens strike back    Chickens Strike Back          Lilly   
93        35    Chickens strike back    Chickens Strike Back          Danny   

          Class   Gender  How often the name was mentioned  \
0    Antagonist        M                                21   
1   Protagonist        M                                26   
2   Protagonist        M                                19   
3   Protagonist        M                                37   
4   Protagonist        F                                75   
..          ...      ...                               ...   
89   Antagonist        M                                 6   
90   Antagonist  Unknown                                17   
91  Protagonist        M                                70   
92  Protagonist        F                                68   
93  Protagonist        M                                39   

                                        matched_characters  \
0                                                 ['Joey']   
1                                 ['Joel_that_red_liquid']   
2                                                ['Timmy']   
3                                                 ['Jota']   
4                                                ['Annie']   
..                                                     ...   
89                                           ['McChicken']   
90                                                ['Eggs']   
91                                 ['Said_Rocco', 'ROCCO']   
92                        ['Lilly_Leo', 'my_friend_Lilly']   
93  ['The_Final_Fight_Danny', 'Uncle_Danny', 'Said_Danny']   

                                                                                                                                                                                                                                                 all_pipeline_characters  \
0                                                                                                                                                                                                                 ['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', 'Earth']   
1                                                                                                                                                                                                                 ['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', 'Earth']   
2                                                                ['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emma', '6:00', 'tomorrow_night', 'Annie', 'the_morning', "Mr_Berkensons_'", '6:30', 'our_Thanksgiving', 'the_White_House', 'the_Cafe', 'Jota', 'Government']   
3                                                                ['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emma', '6:00', 'tomorrow_night', 'Annie', 'the_morning', "Mr_Berkensons_'", '6:30', 'our_Thanksgiving', 'the_White_House', 'the_Cafe', 'Jota', 'Government']   
4                                                                ['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emma', '6:00', 'tomorrow_night', 'Annie', 'the_morning', "Mr_Berkensons_'", '6:30', 'our_Thanksgiving', 'the_White_Ho

In [46]:
char_transform = {}
gps = feats.groupby('pipeline_story_name').groups
for story_name, story_rows in gps.items():
    story_char_transform = feats.loc[story_rows, :].set_index('Main character')['matched_characters_list'].to_dict()
    char_transform[story_name] = story_char_transform
char_transform

{'3002': {'Joey': ['Joey'], 'Joel': ['Joel_that_red_liquid']},
 'A leap into the future': {'Timmy': ['Timmy'],
  'Jota': ['Jota'],
  'Annie': ['Annie'],
  'Henry': ['Henry'],
  'Mr. Berkenson': ["Mr_Berkensons_'"]},
 'A new star': {'Halukinumasanluka': ['The_Halukinumasanluka'],
  'Lucy': ['Lucy']},
 'A turtle tale': {'Blair': [], 'Assistant': [], 'Leader': []},
 'Captain Atomicon': {'Psycho': [],
  'Kaysia': ['Car_Kaysia', 'Bye_Kaysia'],
  'Jason Miller': ['Jayson', 'Mr_Jason', "Jason_'"]},
 'Chickens Strike Back': {'Mcchicken': ['McChicken'],
  'Egg': ['Eggs'],
  'Rocco': ['Said_Rocco', 'ROCCO'],
  'Lilly': ['Lilly_Leo', 'my_friend_Lilly'],
  'Danny': ['The_Final_Fight_Danny', 'Uncle_Danny', 'Said_Danny']},
 'Journey  to  Empire  of  the  Reptilian  King': {'Clyde': ['Clyde_in_desperation'],
  'Exavier': ['Exavier_in_franticly', 'Venenum_Exavier'],
  'Rogue': ['this_Rogue']},
 'Mark and The Aliens': {'Wave': ['Said_Wave', 'TIDAL_WAVE'],
  'Psics': ['Said_Psics'],
  'King Tentaclus': 

In [77]:
output_dirpath

'/home/mamille2/storyq/student_scifi/output/'

In [80]:
# Create dataframe with quotes and assertions per character

import json
import pdb

pd.set_option('display.max_colwidth', None)
filtered_output_dirpath = '/home/mamille2/storyq/student_scifi/output_filtered/'
outlines = []
# for story in list(char_transform.keys())[:1]:
for story in sorted(char_transform):
    story_num = story_name2num[story]
    
    # Load quotes
    quote_attribution_fpath = os.path.join(filtered_output_dirpath, 'quote_attribution', f'story{story_num}.quote.json')
    if os.path.exists(quote_attribution_fpath):
        with open(quote_attribution_fpath) as f:
            quotes = json.load(f)
    else:
        quotes = []
        
    # Load assertions
    assertions_fpath = os.path.join(filtered_output_dirpath, 'assertion_extraction', f'story{story_num}.json')
    if os.path.exists(assertions_fpath):
        with open(assertions_fpath) as f:
            assertions = json.load(f)
    else:
        assertions = {}
    
    # Load original assertions (special case)
    orig_assertions_fpath = os.path.join(output_dirpath, 'assertion_extraction', f'story{story_num}.json')
    if os.path.exists(orig_assertions_fpath):
        with open(orig_assertions_fpath) as f:
            assertions_orig = json.load(f)
    else:
        assertions_orig = {}
    
    # Matching characters
    for char in char_transform[story]:
        matching_chars = char_transform[story][char]
        char_quotes = []
        char_assertions = []
        char_assertions_orig = []
        for pipeline_char in matching_chars:
            char_quotes += [[q['quote'] for q in entry['quotes']] for entry in quotes if entry['speaker']==pipeline_char]
            if pipeline_char in assertions:
                char_assertions += assertions[pipeline_char]
            if pipeline_char in assertions_orig:
                char_assertions_orig += assertions_orig[pipeline_char]
        
        outlines.append([story_num, story, char, matching_chars, char_quotes, char_assertions, char_assertions_orig])
    
char_df = pd.DataFrame(outlines, columns=['story_id', 'story_name', 'main_character', 'matched_characters', 'quotes', 'assertions', 'assertions_real_original'])
char_df

story_id                                        story_name main_character  \
0          1                                              3002           Joey   
1          1                                              3002           Joel   
2          2                            A leap into the future          Timmy   
3          2                            A leap into the future           Jota   
4          2                            A leap into the future          Annie   
..       ...                                               ...            ...   
89        31  What would happen if the world stopped spinning?           Luke   
90        18                                  the big surprise       Dr. Fezz   
91        18                                  the big surprise            Sam   
92        18                                  the big surprise         Dillan   
93        18                                  the big surprise       Genevive   

        matched_characters  \
0                   [Joey]   
1   [Joel_that_red_liquid]   
2                  [Timmy]   
3                   [Jota]   
4                  [Annie]   
..                     ...   
89                  [Luke]   
90               [Dr_Fezz]   
91                      []   
92                      []   
93                      []   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [85]:
char_df.loc[0, 'assertions']

[' “ I said my name is <character name="Joey">Joey</character> . ” “ WAIT WATCH OUT ! ” At that moment <character name="Joel_that_red_liquid">Joel</character> saw a car on fire hurdling towards them .',
 ' At the exact moment , <character name="Joel_that_red_liquid">Joel</character> saw the car , <character name="Joel_that_red_liquid">he</character> grabbed <character name="Joey">Joey</character> and ducked . Five minutes later . “ <character name="Joey">Joey</character> , <character name="Joey">Joey</character> Where are you . ” “ I ‘m over here ” joey said in a very faint voice .',
 ' “ Help I got stuck under the car ” <character name="Joel_that_red_liquid">Joel</character> started running towards <character name="Joey">Joey ’s</character> screams for help .',
 ' So <character name="Joel_that_red_liquid">he</character> pushed the car with all <character name="Joel_that_red_liquid">his</character> might and <character name="Joel_that_red_liquid">he</character> got it off joey . “ <cha

In [86]:
char_df.loc[0, 'assertions_real_original']

[' “ I said my name is <character name="Joey">Joey</character> . ” “ WAIT WATCH OUT ! ” At that moment <character name="Joel_that_red_liquid">Joel</character> saw a car on fire hurdling towards them .',
 ' At the exact moment , <character name="Joel_that_red_liquid">Joel</character> saw the car , <character name="Joel_that_red_liquid">he</character> grabbed <character name="Joey">Joey</character> and ducked . Five minutes later . “ <character name="Joey">Joey</character> , <character name="Joey">Joey</character> Where are you . ” “ I ‘m over here ” joey said in a very faint voice .',
 ' “ Help I got stuck under the car ” <character name="Joel_that_red_liquid">Joel</character> started running towards <character name="Joey">Joey ’s</character> screams for help .',
 ' So <character name="Joel_that_red_liquid">he</character> pushed the car with all <character name="Joel_that_red_liquid">his</character> might and <character name="Joel_that_red_liquid">he</character> got it off joey . “ <cha

In [81]:
# Merge into existing main character features
merged = pd.merge(feats, char_df, left_on=['story_id', 'Main character'], right_on=['story_id', 'main_character'],
                 suffixes=('_original', '_filtered'))
merged

story_id                   Story     pipeline_story_name Main character  \
0          1                    3002                    3002           Joey   
1          1                    3002                    3002           Joel   
2          2  A leap into the future  A leap into the future          Timmy   
3          2  A leap into the future  A leap into the future           Jota   
4          2  A leap into the future  A leap into the future          Annie   
..       ...                     ...                     ...            ...   
89        35    Chickens strike back    Chickens Strike Back      Mcchicken   
90        35    Chickens strike back    Chickens Strike Back            Egg   
91        35    Chickens strike back    Chickens Strike Back          Rocco   
92        35    Chickens strike back    Chickens Strike Back          Lilly   
93        35    Chickens strike back    Chickens Strike Back          Danny   

          Class   Gender  How often the name was mentioned  \
0    Antagonist        M                                21   
1   Protagonist        M                                26   
2   Protagonist        M                                19   
3   Protagonist        M                                37   
4   Protagonist        F                                75   
..          ...      ...                               ...   
89   Antagonist        M                                 6   
90   Antagonist  Unknown                                17   
91  Protagonist        M                                70   
92  Protagonist        F                                68   
93  Protagonist        M                                39   

                               matched_characters_original  \
0                                                 ['Joey']   
1                                 ['Joel_that_red_liquid']   
2                                                ['Timmy']   
3                                                 ['Jota']   
4                                                ['Annie']   
..                                                     ...   
89                                           ['McChicken']   
90                                                ['Eggs']   
91                                 ['Said_Rocco', 'ROCCO']   
92                        ['Lilly_Leo', 'my_friend_Lilly']   
93  ['The_Final_Fight_Danny', 'Uncle_Danny', 'Said_Danny']   

                                                                                                                                                                                                                                                 all_pipeline_characters  \
0                                                                                                                                                                                                                 ['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', 'Earth']   
1                                                                                                                                                                                                                 ['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', 'Earth']   
2                                                                ['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emma', '6:00', 'tomorrow_night', 'Annie', 'the_morning', "Mr_Berkensons_'", '6:30', 'our_Thanksgiving', 'the_White_House', 'the_Cafe', 'Jota', 'Government']   
3                                                                ['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emma', '6:00', 'tomorrow_night', 'Annie', 'the_morning', "Mr_Berkensons_'", '6:30', 'our_Thanksgiving', 'the_White_House', 'the_Cafe', 'Jota', 'Government']   
4                                                                ['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emma', '6:00', 'tomorrow_night', 'Annie', 'the_morning', "Mr_Berkensons_'", '6:30', 'our_Thanksgiving', 'the_White_Ho

In [66]:
merged['quotes_match'] = [a==str(b) for a,b in zip(merged['quotes_original'], merged['quotes_filtered'])]
len(merged) - sum(merged['quotes_match'])

5

In [91]:
merged['assertions_match'] = [str(a)==str(b) for a,b in zip(merged['assertions_real_original'], merged['assertions_filtered'])]
len(merged) - sum(merged['assertions_match'])

8

In [87]:
# Arrange, save out final df
merged.columns

Index(['story_id', 'Story', 'pipeline_story_name', 'Main character', 'Class',
       'Gender', 'How often the name was mentioned',
       'matched_characters_original', 'all_pipeline_characters',
       'quotes_original', 'assertions_original', 'matched_characters_list',
       'story_name', 'main_character', 'matched_characters_filtered',
       'quotes_filtered', 'assertions_filtered', 'assertions_real_original',
       'assertions_match'],
      dtype='object')

In [70]:
feats.columns

Index(['story_id', 'Story', 'pipeline_story_name', 'Main character', 'Class',
       'Gender', 'How often the name was mentioned', 'matched_characters',
       'all_pipeline_characters', 'quotes', 'assertions',
       'matched_characters_list'],
      dtype='object')

In [89]:
merged.rename(columns={'matched_characters_list': 'matched_characters'}, inplace=True)

In [94]:
selected_cols = feats.columns.tolist()[:-3] + ['quotes_original', 'quotes_filtered', 'assertions_real_original', 'assertions_filtered']
outpath = '/home/mamille2/storyq/student_scifi/main_character_filtered_features.csv'
merged[selected_cols].rename(columns={'assertions_real_original': 'assertions_original'}).to_csv(outpath)

In [95]:
merged[selected_cols]

story_id                   Story     pipeline_story_name Main character  \
0          1                    3002                    3002           Joey   
1          1                    3002                    3002           Joel   
2          2  A leap into the future  A leap into the future          Timmy   
3          2  A leap into the future  A leap into the future           Jota   
4          2  A leap into the future  A leap into the future          Annie   
..       ...                     ...                     ...            ...   
89        35    Chickens strike back    Chickens Strike Back      Mcchicken   
90        35    Chickens strike back    Chickens Strike Back            Egg   
91        35    Chickens strike back    Chickens Strike Back          Rocco   
92        35    Chickens strike back    Chickens Strike Back          Lilly   
93        35    Chickens strike back    Chickens Strike Back          Danny   

          Class   Gender  How often the name was mentioned  \
0    Antagonist        M                                21   
1   Protagonist        M                                26   
2   Protagonist        M                                19   
3   Protagonist        M                                37   
4   Protagonist        F                                75   
..          ...      ...                               ...   
89   Antagonist        M                                 6   
90   Antagonist  Unknown                                17   
91  Protagonist        M                                70   
92  Protagonist        F                                68   
93  Protagonist        M                                39   

                                  matched_characters  \
0                                             [Joey]   
1                             [Joel_that_red_liquid]   
2                                            [Timmy]   
3                                             [Jota]   
4                                            [Annie]   
..                                               ...   
89                                       [McChicken]   
90                                            [Eggs]   
91                               [Said_Rocco, ROCCO]   
92                      [Lilly_Leo, my_friend_Lilly]   
93  [The_Final_Fight_Danny, Uncle_Danny, Said_Danny]   

                                                                                                                                                                                                                                                 all_pipeline_characters  \
0                                                                                                                                                                                                                 ['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', 'Earth']   
1                                                                                                                                                                                                                 ['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', 'Earth']   
2                                                                ['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emma', '6:00', 'tomorrow_night', 'Annie', 'the_morning', "Mr_Berkensons_'", '6:30', 'our_Thanksgiving', 'the_White_House', 'the_Cafe', 'Jota', 'Government']   
3                                                                ['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emma', '6:00', 'tomorrow_night', 'Annie', 'the_morning', "Mr_Berkensons_'", '6:30', 'our_Thanksgiving', 'the_White_House', 'the_Cafe', 'Jota', 'Government']   
4                                                                ['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emma', '6:00', 'tomorrow_night', 'Annie', 'the_morning', "Mr_Berkensons_'", '6:30', 'our_Thanksgiving', 'the_White_House', 'the_Cafe', 'Jota', 'Government']   
..                           

## Load main characters

In [42]:
# Load protagonists and antagonists
import pandas as pd

protagonist_data = pd.read_csv('/home/mamille2/storyq/student_scifi/consistency.csv', encoding='iso-8859-1')
protagonist_data['Main character'] = protagonist_data['Main character'].str.strip()
protagonist_data

,Story,Main character,Class,Gender,How often the name was mentioned
0,Chickens strike back,Danny,Protagonist,M,39
1,Chickens strike back,Lilly,Protagonist,F,68
2,Chickens strike back,Rocco,Protagonist,M,70
3,Chickens strike back,Egg,Antagonist,Unknown,17
4,Chickens strike back,Mcchicken,Antagonist,M,6
...,...,...,...,...,...
97,The pandaries,Pandaries,Protagonist,Unknown,11
98,Strangest things,Conor,Protagonist,M,23
99,Strangest things,Rob,Protagonist,M,13
100,Strangest things,Sam,Protagonist,M,14


In [5]:
# How many main characters per story
gped = protagonist_data.groupby('Story').size()
gped

Story
3002                                             2
A leap into the future                           5
A new star                                       2
A turtle tale                                    3
Captain Atomicon                                 3
Chickens strike back                             5
How Could This Happen to the Moon                4
How it happen                                    4
Journey  to  Empire  of  the  Reptilian  King    3
Mark and The Aliens                              4
Pokemon, the story                               3
Project  115                                     1
Project cor3                                     5
Research Gone Wrong!!!                           5
Strangest things                                 4
Teleportation                                    3
The Aqua Doods                                   1
The Great Sand Demon                             4
The Nightmare In Wonderland                      5
The Squad                

In [6]:
gped.mean()

3.642857142857143

## Load pipeline output

In [82]:
# Compare with character lists
import os

output_dirpath = '/home/mamille2/storyq/student_scifi/output/'
story_dirpath = '/home/mamille2/storyq/student_scifi/stories/'
char_dirpath = os.path.join(output_dirpath, 'char_coref_chars')
pipeline_chars = {}
story_name2num = {} # for later use
for fname in os.listdir(char_dirpath):
    story_num = fname.split('.')[0]
    # Get story name
    story_data = pd.read_csv(os.path.join(story_dirpath, f'{story_num}.csv'))
    story_name = str(story_data['story_name'].values[0])
    story_name2num[story_name] = int(story_num.replace('story', ''))
    # Get names
    with open(os.path.join(char_dirpath, fname)) as f:
        pipeline_chars[story_name] = f.read().splitlines()
pipeline_chars

{'Mark and The Aliens': ['Said_Psics',
  '5_Octosai_per_UFO',
  'LT_Leg_Sunken_Ship',
  'Said_LT_Leg',
  'Alright',
  'Said_King_Tentaclus',
  'Said_Greeno',
  'the_Octosai',
  'the_UFO',
  'Said_Wave',
  'Ok_Said_Mark',
  'Said_Octodon',
  'TIDAL_WAVE',
  'Said_Mr_Octo',
  'the_crab_crew_Pinchy'],
 'Water: the Beginning and the End': ['AMAZIN',
  'the_Tsunami',
  'Alright',
  'a_Tsunami',
  'Bill',
  'Juan',
  'the_Virtual_Testing_Room',
  'The_AquaNator_Model',
  'the_AquaNator',
  'Ok',
  'Pablo_Jumps',
  'Tania',
  'Tommorow'],
 '3002': ['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', 'Earth'],
 'The Squad': ['Chris',
  'its_HORIBBLE',
  'Daniella',
  'Kyle',
  'Nick',
  'Nicholas',
  'Earth',
  'Marsami',
  'Daniela',
  'The_Squad',
  'Camille',
  'Flora'],
 'How it happens': ['mayor_Kevin',
  'North_Korea',
  'Bob',
  'John',
  'the_Earth',
  'Said_Sean',
  'the_UN',
  'General_Mcarthur_gone'],
 'What would happen if the world stopped spinning?': ['the_asteroid_Ceres',
  'Zhang',
  

## Align main characters with pipeline output

In [35]:
story_transform = { # pipeline_name: main_char_story_name
    'Project Cor 3': 'Project cor3',
    'What would happen if the world stopped spinning?': 'What would happenÉ',
    'Chickens Strike Back': 'Chickens strike back',
    'Teleportation SciFi': 'Teleportation',
    'Strangest thing': 'Strangest things',
    'the big surprise': 'The big surprise',
    }

In [84]:
matches = set(main_chars.keys()).intersection(set(pipeline_chars.keys()))
mismatches_main_chars = set(main_chars.keys()) - matches
mismatches_pipeline_chars = set(pipeline_chars.keys()) - matches
print(mismatches_main_chars)
print(mismatches_pipeline_chars)

{'Project cor3', 'The big surprise', 'What would happenÉ', 'Chickens strike back', 'Strangest things', 'Teleportation', 'How it happen', 'How Could This Happen to the Moon'}
{'Overwatch', 'Chickens Strike Back', 'What would happen if the world stopped spinning?', 'Teleportation SciFi', 'Strangest thing', 'How it happens', 'Essent Koeln', 'The life of Larry', 'the big surprise', 'The story in the space', 'Project Cor 3'}


In [85]:
# Display character lists
main_chars = protagonist_data.set_index('Main character').groupby('Story').groups
for story in pipeline_chars:
    print(story)
    print('Main characters:')
    if story in main_chars:
        print('\n'.join([f'\t{char}' for char in main_chars[story]]))
    elif story in story_transform:
        print('\n'.join([f'\t{char}' for char in main_chars[story_transform[story]]]))
    else:
        print('no match')
        
    print('Pipeline characters:')
    print('\n'.join([f'\t{char}' for char in pipeline_chars[story]]))
    print()

Mark and The Aliens
Main characters:
	Mark
	King Tentaclus
	Psics
	Wave
Pipeline characters:
	Said_Psics
	5_Octosai_per_UFO
	LT_Leg_Sunken_Ship
	Said_LT_Leg
	Alright
	Said_King_Tentaclus
	Said_Greeno
	the_Octosai
	the_UFO
	Said_Wave
	Ok_Said_Mark
	Said_Octodon
	TIDAL_WAVE
	Said_Mr_Octo
	the_crab_crew_Pinchy

Water: the Beginning and the End
Main characters:
	Juan
	Bill
	Tania
	Pablo
Pipeline characters:
	AMAZIN
	the_Tsunami
	Alright
	a_Tsunami
	Bill
	Juan
	the_Virtual_Testing_Room
	The_AquaNator_Model
	the_AquaNator
	Ok
	Pablo_Jumps
	Tania
	Tommorow

3002
Main characters:
	Joel
	Joey
Pipeline characters:
	Joey
	Joel_that_red_liquid
	Tia_Lopez
	Earth

The Squad
Main characters:
	Chris
	Kyle
	Camille
	Nicholas/Nick
	Bryan
	Daniela
Pipeline characters:
	Chris
	its_HORIBBLE
	Daniella
	Kyle
	Nick
	Nicholas
	Earth
	Marsami
	Daniela
	The_Squad
	Camille
	Flora

How it happens
Main characters:
no match
Pipeline characters:
	mayor_Kevin
	North_Korea
	Bob
	John
	the_Earth
	Said_Sean
	the_UN
	Gene

In [86]:
# Create character alignment
chars_transform = {} # story: {main_char: [pipeline_chars]}
for story, pipeline_chars_list in pipeline_chars.items():
    if story in main_chars:
        main_chars_story = main_chars[story]
    elif story in story_transform:
        main_chars_story = main_chars[story_transform[story]]
    else:
        continue
        
    chars_transform[story] = {}
    for char in main_chars_story:
        if ' ' in char:
            chars_transform[story][char] = [c for c in pipeline_chars_list if char.strip().replace(' ', '_').replace('.','').lower() in c.lower()]
        else:
            chars_transform[story][char] = [c for c in pipeline_chars_list if char.strip().lower() in c.lower()]
            
# Manual edits
chars_transform['The Squad']['Nicholas/Nick'] = ['Nicholas', 'Nick']
chars_transform['Captain Atomicon']['Jason Miller'] = ['Jayson', 'Mr_Jason', "Jason_'"]
chars_transform

{'Mark and The Aliens': {'Mark': ['Ok_Said_Mark'],
  'King Tentaclus': ['Said_King_Tentaclus'],
  'Psics': ['Said_Psics'],
  'Wave': ['Said_Wave', 'TIDAL_WAVE']},
 'Water: the Beginning and the End': {'Juan': ['Juan'],
  'Bill': ['Bill'],
  'Tania': ['Tania'],
  'Pablo': ['Pablo_Jumps']},
 '3002': {'Joel': ['Joel_that_red_liquid'], 'Joey': ['Joey']},
 'The Squad': {'Chris': ['Chris'],
  'Kyle': ['Kyle'],
  'Camille': ['Camille'],
  'Nicholas/Nick': ['Nicholas', 'Nick'],
  'Bryan': [],
  'Daniela': ['Daniela']},
 'What would happen if the world stopped spinning?': {'Luke': ['Luke'],
  'Nathan': ['The_second_Nathan'],
  'Zhang': ['Zhang'],
  'monster': []},
 'Journey  to  Empire  of  the  Reptilian  King': {'Exavier': ['Exavier_in_franticly',
   'Venenum_Exavier'],
  'Rogue': ['this_Rogue'],
  'Clyde': ['Clyde_in_desperation']},
 'Pokemon, the story': {'Magerna': ['Magerna'],
  'Darkness': [],
  'Sycamore': ['professor_Sycamore']},
 'A new star': {'Lucy': ['Lucy'],
  'Halukinumasanluka':

In [78]:
# Count characters in chars_transform
total_chars = 0
for story, chars in chars_transform.items():
    total_chars += len(chars)
total_chars

94

## Retrieve quotes and assertions

In [87]:
import json
import pdb
pd.set_option('display.max_colwidth', None)

outlines = []
# for story in list(chars_transform.keys())[:1]:
for story in sorted(chars_transform):
    story_num = story_name2num[story]
    
    # Load quotes
    with open(os.path.join(output_dirpath, 'quote_attribution', f'story{story_num}.quote.json')) as f:
        quotes = json.load(f)
        
    # Load assertions
    with open(os.path.join(output_dirpath, 'assertion_extraction', f'story{story_num}.json')) as f:
        assertions = json.load(f)
    
    # Matching characters
    for char in chars_transform[story]:
        matching_chars = chars_transform[story][char]
        char_quotes = []
        char_assertions = []
        for pipeline_char in matching_chars:
            char_quotes += [[q['quote'] for q in entry['quotes']] for entry in quotes if entry['speaker']==pipeline_char]
            if pipeline_char in assertions:
                char_assertions += assertions[pipeline_char]
        
        outlines.append([story_num, story, char, matching_chars, pipeline_chars[story], char_quotes, char_assertions])
    
char_df = pd.DataFrame(outlines, columns=['story_id', 'story_name', 'main_character', 'matched_characters', 'all_pipeline_characters', 'quotes', 'assertions'])
char_df

story_id                                        story_name main_character  \
0          1                                              3002           Joel   
1          1                                              3002           Joey   
2          2                            A leap into the future           Jota   
3          2                            A leap into the future          Annie   
4          2                            A leap into the future          Timmy   
..       ...                                               ...            ...   
89        31  What would happen if the world stopped spinning?        monster   
90        18                                  the big surprise       Dr. Fezz   
91        18                                  the big surprise            Sam   
92        18                                  the big surprise         Dillan   
93        18                                  the big surprise       Genevive   

        matched_characters  \
0   [Joel_that_red_liquid]   
1                   [Joey]   
2                   [Jota]   
3                  [Annie]   
4                  [Timmy]   
..                     ...   
89                      []   
90               [Dr_Fezz]   
91                      []   
92                      []   
93                      []   

                                                                                                                                                    all_pipeline_characters  \
0                                                                                                                            [Joey, Joel_that_red_liquid, Tia_Lopez, Earth]   
1                                                                                                                            [Joey, Joel_that_red_liquid, Tia_Lopez, Earth]   
2   [Timmy, Henry, Ok, the_whole_day, Emma, 6:00, tomorrow_night, Annie, the_morning, Mr_Berkensons_', 6:30, our_Thanksgiving, the_White_House, the_Cafe, Jota, Government]   
3   [Timmy, Henry, Ok, the_whole_day, Emma, 6:00, tomorrow_night, Annie, the_morning, Mr_Berkensons_', 6:30, our_Thanksgiving, the_White_House, the_Cafe, Jota, Government]   
4   [Timmy, Henry, Ok, the_whole_day, Emma, 6:00, tomorrow_night, Annie, the_morning, Mr_Berkensons_', 6:30, our_Thanksgiving, the_White_House, the_Cafe, Jota, Government]   
..                                                                                                                                                                      ...   
89                                                                                                          [the_asteroid_Ceres, Zhang, Luke, The_second_Nathan, the_Earth]   
90                                                                                       [Genavive_Sleeman, the_White_House, The_Big_Surprise, the_Fezzgen, Earth, Dr_Fezz]   
91                                                                                       [Genavive_Sleeman, the_White_House, The_Big_Surprise, the_Fezzgen, Earth, Dr_Fezz]   
92                                                                                       [Genavive_Sleeman, the_White_House, The_Big_Surprise, the_Fezzgen, Earth, Dr_Fezz]   
93                                                                                       [Genavive_Sleeman, the_White_House, The_Big_Surprise, the_Fezzgen, Earth, Dr_Fezz]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

## Merge with main character doc

In [95]:
story_transform_reversed = {val: key for key,val in story_transform.items()}
protagonist_data['pipeline_story_name'] = protagonist_data['Story'].map(lambda x: story_transform_reversed.get(x, x))
protagonist_data

,Story,Main character,Class,Gender,How often the name was mentioned,pipeline_story_name
0,Chickens strike back,Danny,Protagonist,M,39,Chickens Strike Back
1,Chickens strike back,Lilly,Protagonist,F,68,Chickens Strike Back
2,Chickens strike back,Rocco,Protagonist,M,70,Chickens Strike Back
3,Chickens strike back,Egg,Antagonist,Unknown,17,Chickens Strike Back
4,Chickens strike back,Mcchicken,Antagonist,M,6,Chickens Strike Back
...,...,...,...,...,...,...
97,The pandaries,Pandaries,Protagonist,Unknown,11,The pandaries
98,Strangest things,Conor,Protagonist,M,23,Strangest thing
99,Strangest things,Rob,Protagonist,M,13,Strangest thing
100,Strangest things,Sam,Protagonist,M,14,Strangest thing


In [96]:
merged = pd.merge(protagonist_data, char_df, left_on=['pipeline_story_name', 'Main character'], right_on=['story_name', 'main_character'])
merged

Story Main character        Class   Gender  \
0   Chickens strike back          Danny  Protagonist        M   
1   Chickens strike back          Lilly  Protagonist        F   
2   Chickens strike back          Rocco  Protagonist        M   
3   Chickens strike back            Egg   Antagonist  Unknown   
4   Chickens strike back      Mcchicken   Antagonist        M   
..                   ...            ...          ...      ...   
89         The pandaries      Pandaries  Protagonist  Unknown   
90      Strangest things          Conor  Protagonist        M   
91      Strangest things            Rob  Protagonist        M   
92      Strangest things            Sam  Protagonist        M   
93      Strangest things           Jeff   Antagonist        M   

    How often the name was mentioned   pipeline_story_name  story_id  \
0                                 39  Chickens Strike Back        35   
1                                 68  Chickens Strike Back        35   
2                                 70  Chickens Strike Back        35   
3                                 17  Chickens Strike Back        35   
4                                  6  Chickens Strike Back        35   
..                               ...                   ...       ...   
89                                11         The pandaries        24   
90                                23       Strangest thing        16   
91                                13       Strangest thing        16   
92                                14       Strangest thing        16   
93                                21       Strangest thing        16   

              story_name main_character  \
0   Chickens Strike Back          Danny   
1   Chickens Strike Back          Lilly   
2   Chickens Strike Back          Rocco   
3   Chickens Strike Back            Egg   
4   Chickens Strike Back      Mcchicken   
..                   ...            ...   
89         The pandaries      Pandaries   
90       Strangest thing          Conor   
91       Strangest thing            Rob   
92       Strangest thing            Sam   
93       Strangest thing           Jeff   

                                  matched_characters  \
0   [The_Final_Fight_Danny, Uncle_Danny, Said_Danny]   
1                       [Lilly_Leo, my_friend_Lilly]   
2                                [Said_Rocco, ROCCO]   
3                                             [Eggs]   
4                                        [McChicken]   
..                                               ...   
89                 [Some_Pandaries, the_Pandaries_']   
90                                   [her_son_Conor]   
91                                             [Rob]   
92                                             [Sam]   
93                        [the_trader_Jeff, Jeffs_']   

                                                                                                                                                                                                         all_pipeline_characters  \
0   [The_Final_Fight_Danny, Said_Rocco, Earth, Dr_Larry, the_ECA, the_Pups, Lilly_Leo, Julia, the_Uncles, Uncle_Larry, Uncle_Danny, Elsa, my_friend_Lilly, New_York_City, Eggs, McChicken, Romeo, ROCCO, Said_Danny, The_Leader]   
1   [The_Final_Fight_Danny, Said_Rocco, Earth, Dr_Larry, the_ECA, the_Pups, Lilly_Leo, Julia, the_Uncles, Uncle_Larry, Uncle_Danny, Elsa, my_friend_Lilly, New_York_City, Eggs, McChicken, Romeo, ROCCO, Said_Danny, The_Leader]   
2   [The_Final_Fight_Danny, Said_Rocco, Earth, Dr_Larry, the_ECA, the_Pups, Lilly_Leo, Julia, the_Uncles, Uncle_Larry, Uncle_Danny, Elsa, my_friend_Lilly, New_York_City, Eggs, McChicken, Romeo, ROCCO, Said_Danny, The_Leader]   
3   [The_Final_Fight_Danny, Said_Rocco, Earth, Dr_Larry, the_ECA, the_Pups, Lilly_Leo, Julia, the_Uncles, Uncle_Larry, Uncle_Danny, Elsa, my_friend_Lilly, New_York_City, Eggs, McChicken, Romeo, ROCCO, Said_Danny, The_Leader]   
4   [The_Final_Fight_Danny, Said_Rocco, Ear

In [99]:
# Rearrange columns
cols = merged.columns.tolist()
cols = ['story_id', 'Story', 'pipeline_story_name', 'Main character', 'Class', 'Gender', 'How often the name was mentioned', 
       'matched_characters', 'all_pipeline_characters', 'quotes', 'assertions']
outdf = merged[cols]
outdf.sort_values('story_id', inplace=True)
outdf

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


story_id                   Story     pipeline_story_name Main character  \
51         1                    3002                    3002           Joey   
50         1                    3002                    3002           Joel   
81         2  A leap into the future  A leap into the future          Timmy   
79         2  A leap into the future  A leap into the future           Jota   
80         2  A leap into the future  A leap into the future          Annie   
..       ...                     ...                     ...            ...   
4         35    Chickens strike back    Chickens Strike Back      Mcchicken   
3         35    Chickens strike back    Chickens Strike Back            Egg   
2         35    Chickens strike back    Chickens Strike Back          Rocco   
1         35    Chickens strike back    Chickens Strike Back          Lilly   
0         35    Chickens strike back    Chickens Strike Back          Danny   

          Class   Gender  How often the name was mentioned  \
51   Antagonist        M                                21   
50  Protagonist        M                                26   
81  Protagonist        M                                19   
79  Protagonist        M                                37   
80  Protagonist        F                                75   
..          ...      ...                               ...   
4    Antagonist        M                                 6   
3    Antagonist  Unknown                                17   
2   Protagonist        M                                70   
1   Protagonist        F                                68   
0   Protagonist        M                                39   

                                  matched_characters  \
51                                            [Joey]   
50                            [Joel_that_red_liquid]   
81                                           [Timmy]   
79                                            [Jota]   
80                                           [Annie]   
..                                               ...   
4                                        [McChicken]   
3                                             [Eggs]   
2                                [Said_Rocco, ROCCO]   
1                       [Lilly_Leo, my_friend_Lilly]   
0   [The_Final_Fight_Danny, Uncle_Danny, Said_Danny]   

                                                                                                                                                                                                         all_pipeline_characters  \
51                                                                                                                                                                                [Joey, Joel_that_red_liquid, Tia_Lopez, Earth]   
50                                                                                                                                                                                [Joey, Joel_that_red_liquid, Tia_Lopez, Earth]   
81                                                       [Timmy, Henry, Ok, the_whole_day, Emma, 6:00, tomorrow_night, Annie, the_morning, Mr_Berkensons_', 6:30, our_Thanksgiving, the_White_House, the_Cafe, Jota, Government]   
79                                                       [Timmy, Henry, Ok, the_whole_day, Emma, 6:00, tomorrow_night, Annie, the_morning, Mr_Berkensons_', 6:30, our_Thanksgiving, the_White_House, the_Cafe, Jota, Government]   
80                                                       [Timmy, Henry, Ok, the_whole_day, Emma, 6:00, tomorrow_night, Annie, the_morning, Mr_Berkensons_', 6:30, our_Thanksgiving, the_White_House, the_Cafe, Jota, Government]   
..                                                                                                                                                                                                                           ...   
4   [The_Final_Fight_Danny, Said_Rocco, E

In [100]:
outpath = '/home/mamille2/storyq/student_scifi/main_character_features.csv'
outdf.to_csv(outpath, index=False)

# Filter pipeline characters to main characters and ones with quotes and assertions
Output to char list files to re-run assertion extraction and quote attribution

In [1]:
# Load main character featurs
import pandas as pd

feats_path = '/home/mamille2/storyq/student_scifi/main_character_features.csv'
feats = pd.read_csv(feats_path)
feats

,story_id,Story,pipeline_story_name,Main character,Class,Gender,How often the name was mentioned,matched_characters,all_pipeline_characters,quotes,assertions
0,1,3002,3002,Joey,Antagonist,M,21,['Joey'],"['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', ...",[['“ I never got the chance to ask where we ar...,['“Bahhh” “I don’t speak sheep.” Joel said./ “...
1,1,3002,3002,Joel,Protagonist,M,26,['Joel_that_red_liquid'],"['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', ...","[['“ What , where am I ? What year is it ? Wai...",[]
2,2,A leap into the future,A leap into the future,Timmy,Protagonist,M,19,['Timmy'],"['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emm...","[['"" Hey , who wants to go to lunch with me , ...","[""After lunch, they all went back to their dor..."
3,2,A leap into the future,A leap into the future,Jota,Protagonist,M,37,['Jota'],"['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emm...","[['"" 100 by100 ""', '"" , did we make this chip ...","["" When they got home, Annie took the newspape..."
4,2,A leap into the future,A leap into the future,Annie,Protagonist,F,75,['Annie'],"['Timmy', 'Henry', 'Ok', 'the_whole_day', 'Emm...","[['"" Come on , , ""', '"" We are going to be lat...","["" When they got home, Annie took the newspape..."
...,...,...,...,...,...,...,...,...,...,...,...
89,35,Chickens strike back,Chickens Strike Back,Mcchicken,Antagonist,M,6,['McChicken'],"['The_Final_Fight_Danny', 'Said_Rocco', 'Earth...",[],[]
90,35,Chickens strike back,Chickens Strike Back,Egg,Antagonist,Unknown,17,['Eggs'],"['The_Final_Fight_Danny', 'Said_Rocco', 'Earth...","[['"" Either way what we need , is a plan of re...",[]
91,35,Chickens strike back,Chickens Strike Back,Rocco,Protagonist,M,70,"['Said_Rocco', 'ROCCO']","['The_Final_Fight_Danny', 'Said_Rocco', 'Earth...","[['"" Eating chicken is n\'t right , this city ...",[]
92,35,Chickens strike back,Chickens Strike Back,Lilly,Protagonist,F,68,"['Lilly_Leo', 'my_friend_Lilly']","['The_Final_Fight_Danny', 'Said_Rocco', 'Earth...","[['"" HEY ! WATCH IT ! ""', '"" Hey , Where are y...",[]


In [2]:
# Load pipeline character lists
import os

output_dirpath = '/home/mamille2/storyq/student_scifi/output/'
story_dirpath = '/home/mamille2/storyq/student_scifi/stories/'
char_dirpath = os.path.join(output_dirpath, 'char_coref_chars')
pipeline_chars = {}
story_name2num = {} # for later use
for fname in os.listdir(char_dirpath):
    story_num = fname.split('.')[0]
    # Get story name
    story_data = pd.read_csv(os.path.join(story_dirpath, f'{story_num}.csv'))
    story_name = str(story_data['story_name'].values[0])
    story_name2num[story_name] = int(story_num.replace('story', ''))
    # Get names
    with open(os.path.join(char_dirpath, fname)) as f:
        pipeline_chars[story_name] = f.read().splitlines()
pipeline_chars

{'Mark and The Aliens': ['Said_Psics',
  '5_Octosai_per_UFO',
  'LT_Leg_Sunken_Ship',
  'Said_LT_Leg',
  'Alright',
  'Said_King_Tentaclus',
  'Said_Greeno',
  'the_Octosai',
  'the_UFO',
  'Said_Wave',
  'Ok_Said_Mark',
  'Said_Octodon',
  'TIDAL_WAVE',
  'Said_Mr_Octo',
  'the_crab_crew_Pinchy'],
 'Water: the Beginning and the End': ['AMAZIN',
  'the_Tsunami',
  'Alright',
  'a_Tsunami',
  'Bill',
  'Juan',
  'the_Virtual_Testing_Room',
  'The_AquaNator_Model',
  'the_AquaNator',
  'Ok',
  'Pablo_Jumps',
  'Tania',
  'Tommorow'],
 '3002': ['Joey', 'Joel_that_red_liquid', 'Tia_Lopez', 'Earth'],
 'The Squad': ['Chris',
  'its_HORIBBLE',
  'Daniella',
  'Kyle',
  'Nick',
  'Nicholas',
  'Earth',
  'Marsami',
  'Daniela',
  'The_Squad',
  'Camille',
  'Flora'],
 'How it happens': ['mayor_Kevin',
  'North_Korea',
  'Bob',
  'John',
  'the_Earth',
  'Said_Sean',
  'the_UN',
  'General_Mcarthur_gone'],
 'What would happen if the world stopped spinning?': ['the_asteroid_Ceres',
  'Zhang',
  

In [13]:
feats['matched_characters_list'] = feats['matched_characters'].map(lambda x: eval(x))
feats['matched_characters_list']

0                                               [Joey]
1                               [Joel_that_red_liquid]
2                                              [Timmy]
3                                               [Jota]
4                                              [Annie]
                            ...                       
89                                         [McChicken]
90                                              [Eggs]
91                                 [Said_Rocco, ROCCO]
92                        [Lilly_Leo, my_friend_Lilly]
93    [The_Final_Fight_Danny, Uncle_Danny, Said_Danny]
Name: matched_characters_list, Length: 94, dtype: object

In [16]:
# Filter to characters that match main characters
# gped = feats.set_index('pipeline_story_name').groupby('pipeline_story_name')
# gped = feats.groupby('pipeline_story_name').agg({'matched_characters': lambda x: ''.join(x.tolist())})
matched_main_chars = feats.groupby('pipeline_story_name').agg({'matched_characters_list': lambda x: sum(x, [])})
matched_main_chars

,matched_characters_list
pipeline_story_name,
3002,"[Joey, Joel_that_red_liquid]"
A leap into the future,"[Timmy, Jota, Annie, Henry, Mr_Berkensons_']"
A new star,"[The_Halukinumasanluka, Lucy]"
A turtle tale,[]
Captain Atomicon,"[Car_Kaysia, Bye_Kaysia, Jayson, Mr_Jason, Jas..."
Chickens Strike Back,"[McChicken, Eggs, Said_Rocco, ROCCO, Lilly_Leo..."
Journey to Empire of the Reptilian King,"[Clyde_in_desperation, Exavier_in_franticly, V..."
Mark and The Aliens,"[Said_Wave, TIDAL_WAVE, Said_Psics, Said_King_..."
"Pokemon, the story","[professor_Sycamore, Magerna]"


In [19]:
filtered_chars = {story_name: set(char_names) for story_name, char_names in matched_main_chars.to_dict()['matched_characters_list'].items()}
filtered_chars

{'3002': {'Joel_that_red_liquid', 'Joey'},
 'A leap into the future': {'Annie',
  'Henry',
  'Jota',
  "Mr_Berkensons_'",
  'Timmy'},
 'A new star': {'Lucy', 'The_Halukinumasanluka'},
 'A turtle tale': set(),
 'Captain Atomicon': {'Bye_Kaysia',
  'Car_Kaysia',
  "Jason_'",
  'Jayson',
  'Mr_Jason'},
 'Chickens Strike Back': {'Eggs',
  'Lilly_Leo',
  'McChicken',
  'ROCCO',
  'Said_Danny',
  'Said_Rocco',
  'The_Final_Fight_Danny',
  'Uncle_Danny',
  'my_friend_Lilly'},
 'Journey  to  Empire  of  the  Reptilian  King': {'Clyde_in_desperation',
  'Exavier_in_franticly',
  'Venenum_Exavier',
  'this_Rogue'},
 'Mark and The Aliens': {'Ok_Said_Mark',
  'Said_King_Tentaclus',
  'Said_Psics',
  'Said_Wave',
  'TIDAL_WAVE'},
 'Pokemon, the story': {'Magerna', 'professor_Sycamore'},
 'Project  115': set(),
 'Project Cor 3': {'Gavin', 'Levar'},
 'Research Gone Wrong!!!': {'Brad', 'Mr_Kernel', 'Ok_Chad', 'Screamed_Lilly'},
 'Strangest thing': {"Jeffs_'",
  'Rob',
  'Sam',
  'her_son_Conor',
  'th

## Add in characters with both quotes and assertions

In [20]:
# Load quotes and assertions

import json
pd.set_option('display.max_colwidth', None)

for story in sorted(filtered_chars):
    story_num = story_name2num[story]
    
    # Load quotes
    with open(os.path.join(output_dirpath, 'quote_attribution', f'story{story_num}.quote.json')) as f:
        quotes = json.load(f)
        
    # Load assertions
    with open(os.path.join(output_dirpath, 'assertion_extraction', f'story{story_num}.json')) as f:
        assertions = json.load(f)
    
    # Get characters with both quotes and assertions
    quote_chars = set([entry['speaker'] for entry in quotes])
    assertion_chars = set(assertions.keys())
    both = quote_chars | assertion_chars
    filtered_chars[story] |= both
    
filtered_chars

{'3002': {'Earth', 'Joel_that_red_liquid', 'Joey'},
 'A leap into the future': {'Annie',
  'Henry',
  'Jota',
  "Mr_Berkensons_'",
  'Timmy',
  'the_White_House'},
 'A new star': {'Kacey',
  'Lucy',
  'The_Halukinumasanluka',
  'my_teacher_Mrs_Smith'},
 'A turtle tale': {'New_York_City', '^#_%*&'},
 'Captain Atomicon': {'Bye_Kaysia',
  'COYOTES',
  'Car_Kaysia',
  "Jason_'",
  'Jayson',
  'Kysia',
  'Maby',
  'Mr_Jason',
  'Phyco',
  'Saturday',
  'the_Bison',
  'the_skunks'},
 'Chickens Strike Back': {'Eggs',
  'Elsa',
  'Julia',
  'Lilly_Leo',
  'McChicken',
  'ROCCO',
  'Romeo',
  'Said_Danny',
  'Said_Rocco',
  'The_Final_Fight_Danny',
  'Uncle_Danny',
  'my_friend_Lilly'},
 'Journey  to  Empire  of  the  Reptilian  King': {'Clyde_in_desperation',
  'Exavier_in_franticly',
  'Haleq',
  'Muktar',
  'Myzev',
  'Venenum_Exavier',
  'day',
  'this_Rogue'},
 'Mark and The Aliens': {'Ok_Said_Mark',
  'Said_King_Tentaclus',
  'Said_Psics',
  'Said_Wave',
  'TIDAL_WAVE'},
 'Pokemon, the st

In [28]:
# Compare number of filtered characters to all characters
filtered_counts = {story_name: len(char_names) for story_name, char_names in filtered_chars.items()}
print(sorted(filtered_counts.items(), key=lambda x: x[0]))
pipeline_counts = {story_name: len(char_names) for story_name, char_names in pipeline_chars.items()}
print()
print(sorted(pipeline_counts.items(), key=lambda x: x[0]))

[('3002', 3), ('A leap into the future', 6), ('A new star', 4), ('A turtle tale', 2), ('Captain Atomicon', 12), ('Chickens Strike Back', 12), ('Journey  to  Empire  of  the  Reptilian  King', 8), ('Mark and The Aliens', 5), ('Pokemon, the story', 5), ('Project  115', 1), ('Project Cor 3', 4), ('Research Gone Wrong!!!', 8), ('Strangest thing', 7), ('Teleportation SciFi', 3), ('The Aqua Doods', 0), ('The Great Sand Demon', 6), ('The Nightmare In Wonderland', 7), ('The Squad', 8), ('The eclipse ostriches', 4), ('The pandaries', 5), ('The revolution', 6), ('The wormhole to the future', 9), ('Tsunami Terror', 5), ('Water: the Beginning and the End', 7), ('What would happen if the world stopped spinning?', 3), ('the big surprise', 3)]

[('3002', 4), ('A leap into the future', 16), ('A new star', 6), ('A turtle tale', 3), ('Captain Atomicon', 18), ('Chickens Strike Back', 20), ('Essent Koeln', 3), ('How it happens', 8), ('Journey  to  Empire  of  the  Reptilian  King', 15), ('Mark and The Ali

In [30]:
len(filtered_chars)

26

In [29]:
# Write out filtered character list
output_dirpath = '/home/mamille2/storyq/student_scifi/output_filtered/char_coref_chars/'
for story_name, chars in filtered_chars.items():
    story_num = story_name2num[story_name]
    with open(os.path.join(output_dirpath, f'story{story_num}.chars'), 'w') as f:
        for char in sorted(chars):
            f.write(f'{char}\n')

In [34]:
# Remove char_coref_stories that aren't also present in char_coref_chars
story_dirpath = '/home/mamille2/storyq/student_scifi/output_filtered/char_coref_stories'
selected_story_nums = [story_name2num[story_name] for story_name in filtered_chars]
for fname in os.listdir(story_dirpath):
    story_num = int(fname.split('.')[0].replace('story', ''))
    if not story_num in selected_story_nums:
        os.remove(os.path.join(story_dirpath, fname))

# Check/compare pipeline output

In [6]:
# Check coref
import os
import pandas as pd

pd.set_option('display.max_colwidth', None)

output_dirpath = '/home/mamille2/storyq/student_scifi/output/'
for fname in os.listdir(os.path.join(output_dirpath, 'char_coref_stories'))[:2]:
    if fname.endswith('.csv'):
        data = pd.read_csv(os.path.join(output_dirpath, 'char_coref_stories', fname))
data['text_tokenized']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [36]:
# Check quote attribution
import json

output_dirpath = '/home/mamille2/storyq/student_scifi/output/'
filtered_output_dirpath = '/home/mamille2/storyq/student_scifi/output_filtered/'

for fname in os.listdir(os.path.join(output_dirpath, 'quote_attribution'))[:1]:
    with open(os.path.join(output_dirpath, 'quote_attribution', fname)) as f:
        data = json.load(f)
        
for fname in os.listdir(os.path.join(filtered_output_dirpath, 'quote_attribution'))[:1]:
    with open(os.path.join(filtered_output_dirpath, 'quote_attribution', fname)) as f:
        filtered_data = json.load(f)
        
print(len(data))
print(len(filtered_data))

17
15


In [38]:
data_speakers = [entry['speaker'] for entry in data]
data_speakers

['Car_Kaysia',
 'Car_Kaysia',
 'Car_Kaysia',
 'Car_Kaysia',
 'Car_Kaysia',
 'Maby',
 'Car_Kaysia',
 'the_Bison',
 'Jayson',
 'Car_Kaysia',
 'the_Bison',
 'Bye_Kaysia',
 'Bye_Kaysia',
 'the_skunks',
 'the_skunks',
 'Bye_Kaysia',
 'Bye_Kaysia']

In [12]:
# Check assertion extraction
for fname in os.listdir(os.path.join(output_dirpath, 'assertion_extraction'))[:1]:
    print(fname)
    with open(os.path.join(output_dirpath, 'assertion_extraction', fname)) as f:
        data = json.load(f)
        
data

story6.json


{'COYOTES': [' OH MY GOD, WHY DO YOU HAVE COYOTES AND SKUNKS IN A NET?!? WAIT WHY DO YOU HAVE COYOTES AND SKUNKS AT ALL, AND WHAT IS THAT STENCH she screams at me./ I say / "Wait you caught him?'],
 'Phyco': [' Phyco sighs and looks at Kaysia with a pitiful look, / says Phyco./  asked Kysia, phyco nods Kaysia slaps him again without warning./  says Kysia./  says Kaysia.'],
 'Kysia': [' Phyco sighs and looks at Kaysia with a pitiful look, / says Phyco./  asked Kysia, phyco nods Kaysia slaps him again without warning./  says Kysia./  says Kaysia.'],
 'Saturday': [' /  I said./  said Kaysia. /We call the police as an anonymous and leave. /  said Kaysia./  I say.  I ask Kaysia./  asks./  I say. /We stop walking and she looks at me. /  she says./  I say./ She nods and says,, and walk away./ I super speed home and celebrate my huge victory./']}

In [13]:
data.keys()

dict_keys(['COYOTES', 'Phyco', 'Kysia', 'Saturday'])

In [14]:
data['Kysia']

[' Phyco sighs and looks at Kaysia with a pitiful look, / says Phyco./  asked Kysia, phyco nods Kaysia slaps him again without warning./  says Kysia./  says Kaysia.']

# Clean CSVs

In [5]:
# Rename CSVs
import os
csv_dirpath = '/home/mamille2/storyq/shiyan_stories'

for fname in os.listdir(csv_dirpath):
    new_fname = f'{fname.split(" - ")[0]}.csv'
    old_fpath = os.path.join(csv_dirpath, fname)
    new_fpath = os.path.join(csv_dirpath, new_fname)
    os.rename(old_fpath, new_fpath)

In [3]:
# Process CSVs
import os
import pandas as pd
import stanza
from tqdm import tqdm_notebook as tqdm

pd.set_option('display.max_colwidth', None)
nlp = stanza.Pipeline(lang='en', processors='tokenize')
csv_dirpath = '/home/mamille2/storyq/shiyan_stories'

def stanza_tokenize(text):
    doc = nlp(text)
    return ' '.join([token.text for sentence in doc.sentences for token in sentence.tokens])

for fname in tqdm(os.listdir(csv_dirpath)):
    story = pd.read_csv(os.path.join(csv_dirpath,fname))
    
    # Remove slashes
    story['text_tokenized'] = story['text'].str.replace('/', '')

    # Tokenize with Stanza
    story['text_tokenized'] = story['text_tokenized'].map(lambda x: stanza_tokenize(x))
    
    # Save out
    story.to_csv(os.path.join(csv_dirpath, fname), index=False)

2020-09-11 21:27:49 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-09-11 21:27:49 INFO: Use device: cpu
2020-09-11 21:27:49 INFO: Loading: tokenize
2020-09-11 21:27:49 INFO: Done loading processors!


In [4]:
story['text_tokenized']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             January , 25 transmission 538
1                                                                           